In [41]:
import pandas as pd
mammals=pd.read_csv("/home/jimbrootan/Desktop/ML_data/Lasso_reg_mamels.csv",sep='\t')
mammals


,Species,BodyWt,BrainWt,NonDreaming,Dreaming,TotalSleep,LifeSpan,Gestation,Predation,Exposure,Danger
0,Africanelephant,6654.000,5712.0,NaN,NaN,3.3,38.6,645.0,3,5,3
1,Africangiantpouchedrat,1.000,6.6,6.3,2.0,8.3,4.5,42.0,3,1,3
2,ArcticFox,3.385,44.5,NaN,NaN,12.5,14.0,60.0,1,1,1
3,Arcticgroundsquirrel,0.920,5.7,NaN,NaN,16.5,NaN,25.0,5,2,3
4,Asianelephant,2547.000,4603.0,2.1,1.8,3.9,69.0,624.0,3,5,4
...,...,...,...,...,...,...,...,...,...,...,...
57,Treehyrax,2.000,12.3,4.9,0.5,5.4,7.5,200.0,3,1,3
58,Treeshrew,0.104,2.5,13.2,2.6,15.8,2.3,46.0,3,2,2
59,Vervet,4.190,58.0,9.7,0.6,10.3,24.0,210.0,4,3,4
60,Wateropossum,3.500,3.9,12.8,6.6,19.4,3.0,14.0,2,1,1


In [3]:
mammals['squread'] = mammals['BodyWt']**2
mammals['cubed'] = mammals['BodyWt']**3
X=mammals[['BodyWt','squread','cubed']]
X
y=mammals[['BrainWt']]
y

,BrainWt
0,5712.0
1,6.6
2,44.5
3,5.7
4,4603.0
...,...
57,12.3
58,2.5
59,58.0
60,3.9


In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
import numpy as np
from sklearn.linear_model import Lasso,Ridge
    

In [40]:
Xscaled=StandardScaler().fit_transform(X) 
yscaled=StandardScaler().fit_transform(y) #scaled data
X_train,X_test,y_train,y_test=train_test_split(Xscaled,yscaled,test_size=.2,random_state=20) #split the data and get X_train,X_test,y_train,y_test
# X_train.shape
# X_test.shape
lambda_value=np.linspace(0.001,0.2,25)

lambda_grid=[{'alpha':lambda_value}]

model_ridge=Ridge(max_iter=10000)
cv_Ridge=GridSearchCV(estimator=model_ridge,param_grid=lambda_grid,cv=KFold(n_splits=20))
cv_Ridge.fit(X_train,y_train)

model_lasso=Lasso(max_iter=10000)
cv_lasso=GridSearchCV(estimator=model_ridge,param_grid=lambda_grid,cv=KFold(n_splits=20))
cv_lasso.fit(X_train,y_train)

cv_Ridge.best_params_['alpha'],cv_lasso.best_params_['alpha']
best_lambda_Ridge=cv_Ridge.best_params_['alpha']
best_lambda_lasso=cv_lasso.best_params_['alpha']

model_Ridge_best=Ridge(alpha=best_lambda_Ridge,max_iter=10000)
model_Ridge_best.fit(X_train,y_train)
print('best coefficient value for Ridge model',model_Ridge_best.coef_)


Ridge_predict=model_Ridge_best.predict(X_test)
res_Ridge=np.mean((Ridge_predict-y_test)**2)
print('Res of (Ridge)%.4f'%res_Ridge)

model_lasso_best=Lasso(alpha=best_lambda_lasso,max_iter=10000)
model_lasso_best.fit(X_train,y_train)
print('\n')
print("best coefficient value for Lasso :",model_lasso_best.coef_)
Lasso_predict=model_lasso_best.predict(X_test)
Res_lasso=np.mean((Lasso_predict-y_test)**2)
print('Res of (LASSO)%.4f'%Res_lasso)

best coefficient value for Ridge model [[ 1.40582141 -3.4070364   2.83346518]]
Res of (Ridge)0.9954


best coefficient value for Lasso : [ 1.08696217 -0.23687505 -0.01433277]
Res of (LASSO)2.2956
